# Final Project for IANNwTF 2022/23 


Learning to colorize grayscale dog pictures with the Stanford Dog Dataset.  

In [1]:
%load_ext tensorboard

In [ ]:
# prepare data

# go through folders
# make pairs of image + breed
# (not needed for grayscale but might need it later)
# make it a tensorflow dataset
# make it grayscale
# make them same size and fill gaps at the edges with black pixels in original and greyscale
# randomly flip horizontally
# or take different crops from the pictures
# make rgb color scheme to Lab color scheme

In [ ]:
# model

# create the whole autoencoder model
# (steal from https://towardsdatascience.com/image-colorization-using-convolutional-autoencoders-fdabc1cb1dbe )


In [ ]:
# training loop

# log results with tensorboard 
# save model to be able to reuse it

In [ ]:
%tensorboard --logdir logs